![image](how.png)

**"Should I Stay or Should i Go (from the Hospital)"**

In [58]:
import pandas as pd
import numpy as np
from multiColumnLabelEncode import MultiColumnLabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import linear_model
# la otra forma, pero lo mismo: from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [59]:
df = pd.read_csv("//Users/elsa/Desktop/Bootcamp The Bridge/The_Bridge/Ds_Bootcamp_Elsa/Ejercicios/competicion_kaggle/train_set.csv")

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  int64  
 1   1       100000 non-null  int64  
 2   2       100000 non-null  object 
 3   3       100000 non-null  int64  
 4   4       100000 non-null  object 
 5   5       100000 non-null  int64  
 6   6       100000 non-null  object 
 7   7       100000 non-null  object 
 8   8       100000 non-null  object 
 9   9       99967 non-null   float64
 10  10      100000 non-null  int64  
 11  11      98517 non-null   float64
 12  12      100000 non-null  object 
 13  13      100000 non-null  object 
 14  14      100000 non-null  int64  
 15  15      100000 non-null  object 
 16  16      100000 non-null  float64
 17  17      100000 non-null  object 
dtypes: float64(3), int64(6), object(9)
memory usage: 13.7+ MB


In [10]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
1,159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
2,309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
3,279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
4,147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60


In [61]:
class MultiColumnLabelEncoder:
       def __init__(self,columns = None):
           self.columns = columns # array of column names to encode

       def fit(self,X,y=None):
           return self # not relevant here

       def transform(self,X):
           '''
           Transforms columns of X specified in self.columns using
           LabelEncoder(). If no columns specified, transforms all
           columns in X.
           '''
           output = X.copy()
           if self.columns is not None:
               for col in self.columns:
                   output[col] = LabelEncoder().fit_transform(output[col])
           else:
               for colname,col in output.iteritems():
                   output[colname] = LabelEncoder().fit_transform(col)
           return output

       def fit_transform(self,X,y=None):
           return self.fit(X,y).transform(X)

In [62]:
# Transformamos el datafrema df para las columnas que sean string los convierta a números:
encoding_pipeline = Pipeline([
       ('encoding',MultiColumnLabelEncoder(columns=['2','4','6','7','8','12','13','15','17']))
       # add more pipeline steps as needed
])
df_transformacion = encoding_pipeline.fit_transform(df)
df_transformacion

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,161528,6,0,6,0,2,2,2,5,4.0,45810,2.0,2,2,2,2,2817.0,0
1,159472,23,0,6,0,4,2,1,5,2.0,128565,15.0,1,2,4,5,4498.0,2
2,309765,2,2,5,2,2,1,3,5,3.0,46565,5.0,2,2,2,7,4573.0,1
3,279614,32,5,9,1,3,2,3,1,4.0,124546,6.0,0,2,4,1,7202.0,5
4,147791,14,0,1,0,3,2,3,4,2.0,22729,8.0,2,2,2,5,3398.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,237869,12,0,9,1,3,2,2,1,3.0,82914,3.0,0,2,6,5,3966.0,10
99996,254763,28,1,11,0,2,2,2,5,2.0,40026,5.0,2,2,3,2,4005.0,5
99997,69788,6,0,6,0,3,2,1,5,3.0,92346,2.0,1,1,2,3,5215.0,3
99998,204442,32,5,9,1,2,2,3,1,4.0,113798,15.0,1,2,3,4,5092.0,1


In [63]:
# PROBLEMA DE CLASIFICACIÓN
df_transformacion["17"].value_counts()

2     27452
1     24476
3     17311
5     10984
0      7427
4      3668
7      3297
10     2150
8      1470
9       908
6       857
Name: 17, dtype: int64

In [39]:
# esto es necesario para evitar futuros errores pero si quitaráis las columnas con Nan no haría falta
# Nan no vale pero 0 sí
#           df = df.dropna()
# esto quita filas y por tanto no se lo podremos aplicar al dataset de predicción

In [64]:
# quitad la primera columna porque corresponde con el case_id y por tanto no le servirá al modelo para aprender...
# quitar otras columnas es decisión vuestra como futuros data scientists 
df_transformacion.drop("0",1,inplace = True)
# Convierto todos los nan en 0
df_transformacion.fillna(0, inplace=True)

#### 1. Definir X e y

In [65]:
# Para generar la X me decanto por las columnas que más correlación tienen
X = np.array(df_transformacion[["1","5","6","7","9","15","16"]])
y = np.array(df_transformacion['17'])
X.shape

(100000, 7)

In [46]:
y.shape

(100000,)

#### 2. Dividir X_train, X_test, y_train, y_test

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [44]:
X_train

array([[3.100e+01, 2.000e+00, 2.000e+00, 6.472e+03],
       [2.300e+01, 4.000e+00, 2.000e+00, 3.924e+03],
       [4.000e+00, 2.000e+00, 2.000e+00, 5.314e+03],
       ...,
       [1.100e+01, 3.000e+00, 2.000e+00, 4.688e+03],
       [2.700e+01, 2.000e+00, 2.000e+00, 4.182e+03],
       [2.500e+01, 4.000e+00, 3.000e+00, 6.884e+03]])

#### 3. Asignar el modelo (vacío) a una variable
Aquí meteríais los parámetros

In [67]:
# MODELO SVM-->EL DE CLASIFICACIÓN SVC

#3 Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X.reshape(-1, 1))  # cuidado con reshape que a veces no hace falta ponerlo porque es un array
y = sc_y.fit_transform(y.reshape(-1, 1))




In [68]:
from sklearn.svm import SVR
regressor = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)


#### 4. Entrenar el modelo con X_train, y_train

In [69]:
regressor.fit(X_train, y_train)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

#### 5. Predecir con el modelo ya entrenado con X_test/to_pred

In [71]:
predictions = regressor.predict(X_test)
print(predictions)

[ 2.97034488 -0.27673485  1.85052557 ...  1.91646334  5.86923083
 -0.23575569]


#### 6. Sacar métricas, valorar el modelo 

In [73]:
regressor.score(X_test,y_test)

-0.5893359635543591

## Una vez listo el modelo, toca predecir con el dataset de predicción 

In [78]:
to_pred = pd.read_csv("/Users/elsa/Desktop/Bootcamp The Bridge/The_Bridge/Ds_Bootcamp_Elsa/Ejercicios/competicion_kaggle/test_set.csv")

In [79]:
# la primera columna es id_case, deberéis quitarla para hacer la predicción si la habéis quitado para entrenar
# pero guardarla para incluirla en la submission

# IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO
# SI EL ARRAY CON EL QUE HICISTEIS .fit() ERA DE 4 COLUMNAS, PARA .predict() DEBEN SER LAS MISMAS
# SI AL ARRAY CON EL QUE HICISTEIS .fit() LO NORMALIZASTEIS, PARA .predict() DEBÉIS NORMALIZARLO
# TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 133744 FILAS
# SI O SI

to_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,Minor,2,41-50,4778.0
1,208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,Moderate,3,31-40,5734.0
2,305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,Minor,3,71-80,5064.0
3,266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,Extreme,4,31-40,3254.0
4,13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,Minor,3,21-30,4639.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133739,318155,18,d,13,Y,6,anesthesia,Q,B,3.0,123269,25.0,Urgent,Minor,4,51-60,5089.0
133740,144850,27,a,7,Y,2,gynecology,S,C,2.0,1293,8.0,Emergency,Moderate,3,61-70,6713.0
133741,180676,16,c,3,Z,3,gynecology,R,A,3.0,112921,5.0,Trauma,Minor,3,31-40,5326.0
133742,39933,28,b,11,X,4,gynecology,R,F,2.0,585,2.0,Trauma,Extreme,2,31-40,7072.0


In [81]:
# Transformamos el datafrema df para las columnas que sean string los convierta a números:
encoding_pipeline = Pipeline([
       ('encoding',MultiColumnLabelEncoder(columns=['2','4','6','7','8','12','13','15']))
       # add more pipeline steps as needed
])
to_pred_transformacion = encoding_pipeline.fit_transform(to_pred)
to_pred_transformacion

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,314114,19,0,7,1,4,2,3,2,2.0,59278,8.0,0,1,2,4,4778.0
1,208989,15,2,5,2,3,2,3,5,2.0,102253,15.0,1,2,3,3,5734.0
2,305872,17,4,1,0,4,2,2,4,4.0,5828,4.0,0,1,3,7,5064.0
3,266099,3,2,3,2,4,0,2,0,2.0,56642,9.0,2,0,4,3,3254.0
4,13228,6,0,6,0,4,2,2,5,1.0,116266,8.0,0,1,3,2,4639.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133739,318155,18,3,13,1,6,1,1,1,3.0,123269,25.0,2,1,4,5,5089.0
133740,144850,27,0,7,1,2,2,3,2,2.0,1293,8.0,0,2,3,6,6713.0
133741,180676,16,2,3,2,3,2,2,0,3.0,112921,5.0,1,1,3,3,5326.0
133742,39933,28,1,11,0,4,2,2,5,2.0,585,2.0,1,0,2,3,7072.0


In [82]:
to_pred_transformacion.drop("0",1,inplace = True)
# para evitar este error ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
to_pred_transformacion = to_pred_transformacion.fillna(0)

In [86]:
predictions_submit = regressor.predict(to_pred_transformacion[["1","5","9","10","11","14","16"]])
predictions_submit

array([2.96026623, 2.96026623, 2.96026623, ..., 2.96026623, 2.96026623,
       2.96026623])

In [90]:
submission = pd.DataFrame({"id": to_pred["0"], "days": predictions_submit})
submission

,id,days
0,314114,2.960266
1,208989,2.960266
2,305872,2.960266
3,266099,2.960266
4,13228,2.960266
...,...,...
133739,318155,2.960266
133740,144850,2.960266
133741,180676,2.960266
133742,39933,2.960266


In [87]:
sample = pd.read_csv("/Users/elsa/Desktop/Bootcamp The Bridge/The_Bridge/Ds_Bootcamp_Elsa/Ejercicios/competicion_kaggle/sample_submission.csv")

In [88]:
sample.shape

(133744, 2)

In [91]:
if submission.shape == sample.shape:
    if submission.columns.all() == sample.columns.all():
        if submission.id.all() == sample.id.all():
            print("you're ready to submit!")
            submission.to_csv("to_submitSVC-0.5893.csv", index = False)
            # ¡¡¡¡¡¡¡ADD INDEX = FALSE!!!!!!!!!

you're ready to submit!
